<a href="https://colab.research.google.com/github/VedPatwardhan/EuroSATClassification/blob/main/EuroSATClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install pytorch-lightning
!pip install neptune-client
!pip install optuna

     |████████████████████████████████| 924 kB 5.4 MB/s 
     |████████████████████████████████| 282 kB 43.8 MB/s 
     |████████████████████████████████| 829 kB 29.4 MB/s 
     |████████████████████████████████| 123 kB 45.8 MB/s 
     |████████████████████████████████| 636 kB 36.6 MB/s 
     |████████████████████████████████| 1.3 MB 37.3 MB/s 
     |████████████████████████████████| 142 kB 43.4 MB/s 
     |████████████████████████████████| 294 kB 42.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=10a47da74f5ee98c54747ff906e1518cbd24afdf999bb22dac4834429514ba6a
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

# Setup

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchmetrics.functional import accuracy
import neptune.new as neptune
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import io
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip ./drive/MyDrive/Datasets/EuroSAT.zip

Streaming output truncated to the last 5000 lines.
  inflating: 2750/SeaLake/SeaLake_2206.jpg  
  inflating: 2750/SeaLake/SeaLake_1313.jpg  
  inflating: 2750/SeaLake/SeaLake_263.jpg  
  inflating: 2750/SeaLake/SeaLake_967.jpg  
  inflating: 2750/SeaLake/SeaLake_515.jpg  
  inflating: 2750/SeaLake/SeaLake_1465.jpg  
  inflating: 2750/SeaLake/SeaLake_1817.jpg  
  inflating: 2750/SeaLake/SeaLake_2902.jpg  
  inflating: 2750/SeaLake/SeaLake_2570.jpg  
  inflating: 2750/SeaLake/SeaLake_1004.jpg  
  inflating: 2750/SeaLake/SeaLake_174.jpg  
  inflating: 2750/SeaLake/SeaLake_2111.jpg  
  inflating: 2750/SeaLake/SeaLake_2388.jpg  
  inflating: 2750/SeaLake/SeaLake_1948.jpg  
  inflating: 2750/SeaLake/SeaLake_838.jpg  
  inflating: 2750/SeaLake/SeaLake_2738.jpg  
  inflating: 2750/SeaLake/SeaLake_1999.jpg  
  inflating: 2750/SeaLake/SeaLake_2359.jpg  
  inflating: 2750/SeaLake/SeaLake_2660.jpg  
  inflating: 2750/SeaLake/SeaLake_1775.jpg  
  inflating: 2750/SeaLake/SeaLake_605.jpg  
  inflatin

In [ ]:
image_folder = torchvision.datasets.ImageFolder('./2750')
image_folder.samples

[('./2750/AnnualCrop/AnnualCrop_1.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_10.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_100.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1000.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1001.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1002.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1003.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1004.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1005.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1006.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1007.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1008.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1009.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_101.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1010.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1011.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1012.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1013.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1014.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1015.jpg', 0),
 ('./2750/AnnualCrop/AnnualCrop_1016.jpg', 0),
 ('./2750/AnnualCrop

In [ ]:
data = np.array(image_folder.samples)[:, 0]     
targets = np.array(image_folder.samples)[:, 1] 
targets = np.array(targets, dtype=int)
print(data)
print(targets)

['./2750/AnnualCrop/AnnualCrop_1.jpg'
 './2750/AnnualCrop/AnnualCrop_10.jpg'
 './2750/AnnualCrop/AnnualCrop_100.jpg' ...
 './2750/SeaLake/SeaLake_997.jpg' './2750/SeaLake/SeaLake_998.jpg'
 './2750/SeaLake/SeaLake_999.jpg']
[0 0 0 ... 9 9 9]


# Dataset

In [ ]:
class EuroSATDataset(torch.utils.data.Dataset):
  def __init__(self, data, targets, root_dir, transform):
    self.data = data
    self.targets = targets
    self.root_dir = root_dir
    self.transform = transform
  
  def __len__(self):
    return self.data.shape[0]
  
  def __getitem__(self, index):
    img_path = self.data[index]
    image = io.imread(img_path)
    label = self.targets[index]
    image = self.transform(image)
    return torch.tensor(image), torch.tensor(label)

# Model

In [ ]:
class EuroSATNet(pl.LightningModule):
  def __init__(self, data, targets, root_dir, batch_size, lr, optim_algo='SGD', model):
    super().__init__()
    self.model = model
    self.loss = nn.CrossEntropyLoss()
    self.transform_train = transforms.Compose(
        [
         transforms.ToPILImage(),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    self.batch_size = batch_size
    self.lr = lr
    self.optim_algo = optim_algo
    self.dataset = EuroSATDataset(data, targets, root_dir, self.transform_train)
    self.train_set, self.val_set = torch.utils.data.random_split(self.dataset, [20000, 7000])

  def forward(self, x):
    logits = self.model(x)
    return logits
  
  def configure_optimizers(self):
    if self.optim_algo == 'SGD':
      optimizer = optim.SGD(self.parameters(), lr=self.lr)
    elif self.optim_algo == 'Adam':
      optimizer = optim.Adam(self.parameters(), lr=self.lr)
    return optimizer
  
  def train_dataloader(self):
    train_loader = torch.utils.data.DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
    return train_loader
  
  def val_dataloader(self):
    val_loader = torch.utils.data.DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
    return val_loader
  
  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    J = self.loss(logits, y)
    acc = accuracy(logits, y)
    run['train_acc'].log(acc)
    run['loss'].log(J)
    pbar = {'train_acc': acc}
    return {'loss': J, 'progress_bar': pbar}
  
  def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    J = self.loss(logits, y)
    acc = accuracy(logits, y)
    run['val_acc'].log(acc)
    pbar = {'val_acc': acc}
    return {'loss': J, 'progress_bar': pbar}
  
  def validation_epoch_end(self, val_step_outputs):
    avg_val_loss = torch.tensor([x['loss'] for x in val_step_outputs]).mean()
    avg_val_acc = torch.tensor([x['progress_bar']['val_acc'] for x in val_step_outputs]).mean()
    run['avg_val_acc'].log(avg_val_acc)
    run['avg_val_loss'].log(avg_val_loss)
    pbar = {"avg_val_acc": avg_val_acc}
    return {'val_loss': avg_val_loss, 'progress_bar': pbar}

# Training

In [ ]:
batch_size = 32
lr = 0.05

In [ ]:
alex_net = torchvision.models.alexnet()

In [ ]:
alex_net.classifier[-1] = nn.Linear(in_features=4096, out_features=10, bias=True)

In [ ]:
alex_net.features[0] = nn.Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
alex_net.features[3] = nn.Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

In [ ]:
alex_net.classifier[0] = nn.Dropout(p=0.65, inplace=False)
alex_net.classifier[3] = nn.Dropout(p=0.65, inplace=False)

In [ ]:
model = EuroSATNet(data, targets, root_dir='./2750', batch_size=batch_size, lr=lr, model=alex_net)

In [ ]:
trainer = pl.Trainer(progress_bar_refresh_rate=10, max_epochs=30, gpus=1)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | features   | Sequential       | 2.6 M 
1 | classifier | Sequential       | 54.6 M
2 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
57.2 M    Trainable params
0         Non-trainable params
57.2 M    Total params
228.840   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]